<a href="https://colab.research.google.com/github/hjwang0086/Coursera_Capstone/blob/main/Assignment2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

<h2>Scrape from URL</h2>

In [19]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table")

table_contents = []
for row in table.findAll("td"):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
table_contents[:3]

[{'Borough': 'North York', 'Neighborhood': 'Parkwoods', 'PostalCode': 'M3A'},
 {'Borough': 'North York',
  'Neighborhood': 'Victoria Village',
  'PostalCode': 'M4A'},
 {'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront',
  'PostalCode': 'M5A'}]

<h2>Convert into DataFrame</h2>

In [20]:
df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                    'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                    'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                    'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [21]:
df.shape

(103, 3)

<h2>Get coordinates from CSV</h2>

In [22]:
import io

url = "http://cocl.us/Geospatial_data"
html = requests.get(url).content
df2 = pd.read_csv(io.StringIO(html.decode('utf-8')))
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
temp = []
for code, borough, neighborhood in df.values:
  latitude, longitude = df2[df2["Postal Code"]==code][["Latitude", "Longitude"]].values[0]
  temp.append({"Postal Code": code, "Borough": borough, "Neighborhood": neighborhood,
          "Latitude": latitude, "Longitude": longitude})
df = pd.DataFrame(temp)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
